# Creating DataFrames
We will create `DataFrame` objects from other data structures in Python, by reading in a CSV file, and by querying a database.

## Imports

In [1]:
import datetime
import numpy as np
import pandas as pd

## Creating a `Series`

In [2]:
np.random.seed(0) # set a seed for reproducibility
pd.Series(np.random.rand(5), name='random')

0    0.548814
1    0.715189
2    0.602763
3    0.544883
4    0.423655
Name: random, dtype: float64

## Creating a `DataFrame` from a `Series`
Use the `to_frame()` method:

In [3]:
pd.Series(np.linspace(0, 10, num=5)).to_frame()

,0
0,0.0
1,2.5
2,5.0
3,7.5
4,10.0


## Creating a `DataFrame` from Python Data Structures
### From a dictionary of list-like structures
The dictionary values can be lists, numpy arrays, etc. as long as they have length (generators don't have length so we can't use them here):

In [4]:
np.random.seed(0) # set seed so result is always the same
pd.DataFrame(
    {
        'date': pd.date_range(
            end=datetime.date(2019, 4, 21),
            freq='1D',
            periods=5
        ),
        'random': np.random.rand(5),
        'text': ['hot', 'warm', 'cool', 'cold', None],
        'truth': [np.random.choice([True, False]) for _ in range(5)]
    }
)

,date,random,text,truth
0,2019-04-17,0.548814,hot,False
1,2019-04-18,0.715189,warm,True
2,2019-04-19,0.602763,cool,True
3,2019-04-20,0.544883,cold,False
4,2019-04-21,0.423655,None,True


### From a list of dictionaries

In [5]:
pd.DataFrame([
    {'mag' : 5.2, 'place' : 'California'},
    {'mag' : 1.2, 'place' : 'Alaska'},
    {'mag' : 0.2, 'place' : 'California'},
])

,mag,place
0,5.2,California
1,1.2,Alaska
2,0.2,California


### From a list of tuples
This is equivalent to using `pd.DataFrame.from_records()`:

In [6]:
list_of_tuples = [(n, n**2, n**3) for n in range(5)]
list_of_tuples

[(0, 0, 0), (1, 1, 1), (2, 4, 8), (3, 9, 27), (4, 16, 64)]

In [7]:
pd.DataFrame(
    list_of_tuples, 
    columns=['n', 'n_squared', 'n_cubed']
)

,n,n_squared,n_cubed
0,0,0,0
1,1,1,1
2,2,4,8
3,3,9,27
4,4,16,64


### From a numpy array

In [8]:
pd.DataFrame(
    np.array([
        [0, 0, 0],
        [1, 1, 1],
        [2, 4, 8],
        [3, 9, 27],
        [4, 16, 64]
    ]), columns=['n', 'n_squared', 'n_cubed']
)

,n,n_squared,n_cubed
0,0,0,0
1,1,1,1
2,2,4,8
3,3,9,27
4,4,16,64


## Creating a `DataFrame` by Reading in a CSV File

In [9]:
df = pd.read_csv('data/earthquakes.csv')

## Writing a `DataFrame` to a CSV File
Note that the index of `df` is just row numbers, so we don't want to keep it.

In [10]:
df.to_csv('output.csv', index=False)

## Writing a `DataFrame` to a Database
Note the `if_exists` parameter. By default it will give you an error if you try to write a table that already exists. Here, we don't care if it is overwritten. Lastly, if we are interested in appending new rows, we set that to `'append'`.

In [11]:
import sqlite3

with sqlite3.connect('data/quakes.db') as connection:
    pd.read_csv('data/tsunamis.csv').to_sql(
        'tsunamis', connection, index=False, if_exists='replace'
    )

## Creating a `DataFrame` by Querying a Database
Using a SQLite database. Otherwise you need to install [SQLAlchemy](https://www.sqlalchemy.org/).

In [12]:
import sqlite3

with sqlite3.connect('data/quakes.db') as connection:
    tsunamis = pd.read_sql('SELECT * FROM tsunamis', connection)

tsunamis.head()

,alert,type,title,place,magType,mag,time
0,None,earthquake,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...","165km NNW of Flying Fish Cove, Christmas Island",mww,5.0,1539459504090
1,green,earthquake,"M 6.7 - 262km NW of Ozernovskiy, Russia","262km NW of Ozernovskiy, Russia",mww,6.7,1539429023560
2,green,earthquake,"M 5.6 - 128km SE of Kimbe, Papua New Guinea","128km SE of Kimbe, Papua New Guinea",mww,5.6,1539312723620
3,green,earthquake,"M 6.5 - 148km S of Severo-Kuril'sk, Russia","148km S of Severo-Kuril'sk, Russia",mww,6.5,1539213362130
4,green,earthquake,"M 6.2 - 94km SW of Kokopo, Papua New Guinea","94km SW of Kokopo, Papua New Guinea",mww,6.2,1539208835130
